<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np

import time

from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Напишем функцию для получения информации о датасете

def status(data):

    print(data.head())
    print()
    print('Информация о датасете: ')
    print(data.info())
    print('------------------------')
    print('Количество пропусков в данных')
    print(data.isna().sum())
    print('------------------------')
    print('Количество дупликатов в данных:', data.duplicated().sum())
    print('------------------------')
    print(data.describe())

In [4]:
df = pd.read_csv('/data_sets/network_traffic_data.csv')

In [5]:
status(df)

    Destination Port   Flow Duration   Total Fwd Packets  \
0                 80         5480074                   3   
1                443          711977                   9   
2                 53          153398                   2   
3                 53           57660                   1   
4               8446             767                   3   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        1                           12   
1                       10                          703   
2                        2                           80   
3                        1                           46   
4                        1                           43   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                          3950                     267   
2                           224                      40   
3                           128                 

In [6]:
df.columns = df.columns.str.lower()
df.drop_duplicates()

,destination port,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
0,80,5480074,3,1,12,0,6,0,4.000,3.465,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,443,711977,9,10,703,3950,267,0,78.100,103.300,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,153398,2,2,80,224,40,40,40.000,0.000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,53,57660,1,1,46,128,46,46,46.000,0.000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,8446,767,3,1,43,6,31,6,14.336,14.440,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539611,80,11512204,8,5,326,11632,326,0,40.750,115.250,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,DoS GoldenEye
539612,80,11513325,5,5,471,3525,471,0,94.200,210.600,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,DoS GoldenEye
539613,80,11509201,7,6,314,11632,314,0,44.840,118.700,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,DoS GoldenEye
539614,80,11509095,8,5,369,11632,369,0,46.120,130.500,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,DoS GoldenEye


In [7]:
df.corr().style.background_gradient(cmap='coolwarm')

In [8]:
df.columns

Index([' destination port', ' flow duration', ' total fwd packets',
       ' total backward packets', 'total length of fwd packets',
       ' total length of bwd packets', ' fwd packet length max',
       ' fwd packet length min', ' fwd packet length mean',
       ' fwd packet length std', 'bwd packet length max',
       ' bwd packet length min', ' bwd packet length mean',
       ' bwd packet length std', 'flow bytes/s', ' flow packets/s',
       ' flow iat mean', ' flow iat std', ' flow iat max', ' flow iat min',
       'fwd iat total', ' fwd iat mean', ' fwd iat std', ' fwd iat max',
       ' fwd iat min', 'bwd iat total', ' bwd iat mean', ' bwd iat std',
       ' bwd iat max', ' bwd iat min', 'fwd psh flags', ' bwd psh flags',
       ' fwd urg flags', ' bwd urg flags', ' fwd header length',
       ' bwd header length', 'fwd packets/s', ' bwd packets/s',
       ' min packet length', ' max packet length', ' packet length mean',
       ' packet length std', ' packet length variance', '

In [9]:
df.columns = df.columns.str.strip()

In [10]:
# Список столбцов для удаления
columns_to_drop = ["bwd psh flags", "bwd urg flags", "fwd avg bytes/bulk", 
                   "fwd avg packets/bulk", "fwd avg bulk rate", "bwd avg bytes/bulk",
                   "bwd avg packets/bulk", "bwd avg bulk rate"]

# Удаляем столбцы
df = df.drop(columns=columns_to_drop, axis=1)

In [11]:
x = df.drop(columns='label')
y = df['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [12]:
print("Размеры выборок:")
print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

Размеры выборок:
x_train: (404712, 70)
x_test: (134904, 70)
y_train: (404712,)
y_test: (134904,)


In [14]:
# Создаем модель классификатора
model_cat = CatBoostClassifier(verbose=2)  # Параметры могут варьироваться

# Обучаем модель
model_cat.fit(x_train, y_train)

# Предсказываем целевые значения
target_pred = model_cat.predict(x_test)

# Оценка качества модели
report = classification_report(y_test, target_pred)
print(report)

# Оцениваем метрики
precision = precision_score(y_test, target_pred, average='weighted')
recall = recall_score(y_test, target_pred, average='weighted')
f1 = f1_score(y_test, target_pred, average='weighted')
accuracy = accuracy_score(y_test, target_pred)

# Выводим результаты
print(f'Weighted Precision: {precision}')
print(f'Weighted Recall: {recall}')
print(f'Weighted F1 Score: {f1}')
print(f'Accuracy: {accuracy}')

Learning rate set to 0.10747
0:	learn: 1.5041085	total: 1.78s	remaining: 29m 39s
2:	learn: 1.0292415	total: 5.22s	remaining: 28m 53s
4:	learn: 0.7681929	total: 8.74s	remaining: 28m 58s
6:	learn: 0.6005753	total: 12.5s	remaining: 29m 37s
8:	learn: 0.4820466	total: 16s	remaining: 29m 19s
10:	learn: 0.3912770	total: 19.5s	remaining: 29m 15s
12:	learn: 0.3262333	total: 23s	remaining: 29m 8s
14:	learn: 0.2751860	total: 26.9s	remaining: 29m 26s
16:	learn: 0.2341225	total: 31.1s	remaining: 30m
18:	learn: 0.1994787	total: 35.5s	remaining: 30m 30s
20:	learn: 0.1719436	total: 39.1s	remaining: 30m 21s
22:	learn: 0.1486521	total: 42.9s	remaining: 30m 20s
24:	learn: 0.1286477	total: 46.8s	remaining: 30m 23s
26:	learn: 0.1147792	total: 50.4s	remaining: 30m 16s
28:	learn: 0.1022486	total: 53.9s	remaining: 30m 3s
30:	learn: 0.0918912	total: 57.6s	remaining: 30m
32:	learn: 0.0828521	total: 1m 1s	remaining: 30m 5s
34:	learn: 0.0756681	total: 1m 5s	remaining: 30m 2s
36:	learn: 0.0700270	total: 1m 8s	rema

302:	learn: 0.0078520	total: 10m 7s	remaining: 23m 17s
304:	learn: 0.0078360	total: 10m 11s	remaining: 23m 13s
306:	learn: 0.0078118	total: 10m 14s	remaining: 23m 8s
308:	learn: 0.0077731	total: 10m 18s	remaining: 23m 3s
310:	learn: 0.0077468	total: 10m 22s	remaining: 22m 59s
312:	learn: 0.0077174	total: 10m 26s	remaining: 22m 54s
314:	learn: 0.0076986	total: 10m 29s	remaining: 22m 49s
316:	learn: 0.0076718	total: 10m 33s	remaining: 22m 45s
318:	learn: 0.0076539	total: 10m 37s	remaining: 22m 41s
320:	learn: 0.0076339	total: 10m 41s	remaining: 22m 37s
322:	learn: 0.0076223	total: 10m 45s	remaining: 22m 32s
324:	learn: 0.0075951	total: 10m 48s	remaining: 22m 27s
326:	learn: 0.0075677	total: 10m 52s	remaining: 22m 23s
328:	learn: 0.0075420	total: 10m 56s	remaining: 22m 19s
330:	learn: 0.0075245	total: 11m	remaining: 22m 15s
332:	learn: 0.0075129	total: 11m 4s	remaining: 22m 10s
334:	learn: 0.0075024	total: 11m 8s	remaining: 22m 6s
336:	learn: 0.0074723	total: 11m 12s	remaining: 22m 2s
338

598:	learn: 0.0059603	total: 19m 31s	remaining: 13m 4s
600:	learn: 0.0059539	total: 19m 35s	remaining: 13m
602:	learn: 0.0059397	total: 19m 39s	remaining: 12m 56s
604:	learn: 0.0059353	total: 19m 43s	remaining: 12m 52s
606:	learn: 0.0059273	total: 19m 47s	remaining: 12m 48s
608:	learn: 0.0059239	total: 19m 50s	remaining: 12m 44s
610:	learn: 0.0059022	total: 19m 54s	remaining: 12m 40s
612:	learn: 0.0058927	total: 19m 58s	remaining: 12m 36s
614:	learn: 0.0058868	total: 20m 2s	remaining: 12m 32s
616:	learn: 0.0058724	total: 20m 5s	remaining: 12m 28s
618:	learn: 0.0058649	total: 20m 9s	remaining: 12m 24s
620:	learn: 0.0058608	total: 20m 13s	remaining: 12m 20s
622:	learn: 0.0058573	total: 20m 17s	remaining: 12m 16s
624:	learn: 0.0058533	total: 20m 21s	remaining: 12m 12s
626:	learn: 0.0058490	total: 20m 24s	remaining: 12m 8s
628:	learn: 0.0058413	total: 20m 29s	remaining: 12m 4s
630:	learn: 0.0058245	total: 20m 33s	remaining: 12m 1s
632:	learn: 0.0058205	total: 20m 36s	remaining: 11m 57s
634

898:	learn: 0.0052744	total: 29m 4s	remaining: 3m 15s
900:	learn: 0.0052697	total: 29m 8s	remaining: 3m 12s
902:	learn: 0.0052678	total: 29m 11s	remaining: 3m 8s
904:	learn: 0.0052651	total: 29m 15s	remaining: 3m 4s
906:	learn: 0.0052626	total: 29m 19s	remaining: 3m
908:	learn: 0.0052596	total: 29m 23s	remaining: 2m 56s
910:	learn: 0.0052573	total: 29m 27s	remaining: 2m 52s
912:	learn: 0.0052500	total: 29m 31s	remaining: 2m 48s
914:	learn: 0.0052476	total: 29m 35s	remaining: 2m 44s
916:	learn: 0.0052439	total: 29m 39s	remaining: 2m 41s
918:	learn: 0.0052405	total: 29m 42s	remaining: 2m 37s
920:	learn: 0.0052363	total: 29m 46s	remaining: 2m 33s
922:	learn: 0.0052348	total: 29m 50s	remaining: 2m 29s
924:	learn: 0.0052325	total: 29m 54s	remaining: 2m 25s
926:	learn: 0.0052293	total: 29m 58s	remaining: 2m 21s
928:	learn: 0.0052276	total: 30m 1s	remaining: 2m 17s
930:	learn: 0.0052230	total: 30m 6s	remaining: 2m 13s
932:	learn: 0.0052181	total: 30m 9s	remaining: 2m 9s
934:	learn: 0.0052163	

In [26]:
x_test.columns

Index(['destination port', 'flow duration', 'total fwd packets',
       'total backward packets', 'total length of fwd packets',
       'total length of bwd packets', 'fwd packet length max',
       'fwd packet length min', 'fwd packet length mean',
       'fwd packet length std', 'bwd packet length max',
       'bwd packet length min', 'bwd packet length mean',
       'bwd packet length std', 'flow bytes/s', 'flow packets/s',
       'flow iat mean', 'flow iat std', 'flow iat max', 'flow iat min',
       'fwd iat total', 'fwd iat mean', 'fwd iat std', 'fwd iat max',
       'fwd iat min', 'bwd iat total', 'bwd iat mean', 'bwd iat std',
       'bwd iat max', 'bwd iat min', 'fwd psh flags', 'fwd urg flags',
       'fwd header length', 'bwd header length', 'fwd packets/s',
       'bwd packets/s', 'min packet length', 'max packet length',
       'packet length mean', 'packet length std', 'packet length variance',
       'fin flag count', 'syn flag count', 'rst flag count', 'psh flag count',

In [15]:
# Сохраняем модель
model_cat.save_model("loan_dtm_model.cbm")

In [48]:
import requests


application = x_test.iloc[2232].to_dict()


url = "http://192.168.88.215:8888/predict"

# Send request
resp = requests.post(url, json=application)

# Print result
print(resp.json())

{'predictions': [['BENIGN']]}


In [ ]:
# , index=[0])